<a href="https://colab.research.google.com/github/dheerajrathee/IADS_SummerSchool_2019/blob/master/GradientBoostingClassifier_IADS2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive/My Drive/IADS-2019-Tree-Codes
# !pwd

# **Gradient Boosting for Classification**
## 1. IRIS Data






# 1. IRIS DATA


## Dataset for Classification




> **Dataset:**  [Iris dataset](https://scikit-learn.org/stable/datasets/index.html#iris-dataset).



*   **Number of Instances:** 
    *   150 (50 in each of three classes)
*   **Number of Attributes:**
    *   4 numeric, predictive attributes and the class

*   **Attribute Information:**
    *   sepal length in cm
    *   sepal width in cm
    *   petal length in cm
    *   petal width in cm

*   **Classes:**
    *   Setosa (0)
    *   Versicolour (1)
    *   Virginica (2)
    






In [0]:
# Add liberaries 
from sklearn import datasets  # DATA
from sklearn.model_selection import train_test_split # to Split Train-Test data
from sklearn import ensemble # To get Decision Tree 
from sklearn import metrics # To generate evaluation metrices
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score


from sklearn.tree import export_graphviz # exporting the tree structure as dot file
from pydotplus.graphviz import graph_from_dot_data # export png image from dot file
from IPython.display import Image, SVG # Show the image within colab notebook
from graphviz import Source
import matplotlib.pyplot as plt

import pandas as pd # for basic data manipulations 
import numpy as np



### 1. Load Data

In [0]:
#load data and see meta info
iris = datasets.load_iris()
dir(iris)

### 2. Explore Data


In [0]:
# print type and shape of data
print(type(iris.data))
print(type(iris.target))

print(iris.data.shape)
print(iris.target.shape)

### 3. Create Panda Dataframe and do data manipulations

In [0]:
dfCls = pd.DataFrame(iris.data, columns=iris.feature_names)
dfCls.head()

In [0]:
# Add target data to the panda dataframe
dfCls['target'] = iris.target
dfCls.head()

### 4. Split the data for Training and Testing

In [0]:
X_train, X_test, y_train, y_test = train_test_split(dfCls.drop(['target'],axis='columns'), iris.target, test_size=0.2,random_state=0, stratify=iris.target)
print(X_train.shape)
print(X_test.shape)

### 5. Initialise a Gradient Boosting Classifier

In [0]:
gbClassifier = ensemble.GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)



> ***Let's dig into*** **[tree.GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)**




### 6. Model Evaluation on Train data

In [0]:
#perform 10 fold cross validation and plot the CM
CV_predicted = cross_val_predict(gbClassifier, X_train, y_train, cv=10) #CV predicted values (training data)
CV_score = cross_val_score(gbClassifier, X_train, y_train, cv=10) #CV model score (training data)

print("Cross validation Score on train data: ",CV_score.mean())
print("\n")

print("Confusion matrix on CV predictions (train data)")
print(metrics.confusion_matrix(y_train, CV_predicted)) # confusion matrix on CV predictions (train data)
print("\n")

print("Classification report CV predictions (train data)")
print(metrics.classification_report(y_train, CV_predicted, target_names=['Setosa', 'Versicolor', 'Virginica'])) # classification report CV predictions (train data)


### 7. Let's fit the RF model on Training data and perform prediction with the Test data 

In [0]:
gbClassMdl = gbClassifier.fit(X_train,y_train)

y_predicted = gbClassMdl.predict(X_test)

### 8. Model Evaluation on Test Data

In [0]:
mdl_score = gbClassMdl.score(X_test,y_test) #model score (test data)
print ("Model Score on test data:",mdl_score)
print("\n")

print("Confusion matrix (test data)")
print(metrics.confusion_matrix(y_test, y_predicted)) #confusion matrix (test data)
print("\n")

print("Classification report (test data)")
print(metrics.classification_report(y_test, y_predicted, target_names=['Setosa', 'Versicolor', 'Virginica'])) # classification report (test data)

# 2. Mushroom Classification Data (Kaggle)
[See further details](https://www.kaggle.com/uciml/mushroom-classification)



### 1. Load Data

In [0]:
#load data from local drive
mushData = pd.read_csv('mushrooms.csv')

### 2. Explore Data

In [0]:
#print first five rows of the data
mushData.head()

In [0]:
#print size of the data
mushData.shape

In [0]:
#print data attributes
mushData.describe()

In [0]:
#print key informations about the data
mushData.info()


In [0]:
#Chack the class balance
mushData['class'].value_counts()

### 3. Perform data manipulations

In [0]:
from sklearn.preprocessing import LabelEncoder #print first five rows of the data
labelencoder=LabelEncoder()
for col in mushData.columns:
    mushData[col] = labelencoder.fit_transform(mushData[col]) #Transform categrical data to numerical data
 
mushData.head()

In [0]:
mushData.describe()

In [0]:
target = mushData['class'] #get the labels as targets and convert to numpy array
np.array(target, dtype=pd.Series)

### 4. Split the data for Training and Testing

In [0]:
X_train, X_test, y_train, y_test = train_test_split(mushData.drop(['class'],axis='columns'), target, test_size=0.2,random_state=123, stratify=target)
print(X_train.shape)
print(X_test.shape)

### 5. Perform Grid search for getting the best parameters

In [0]:

from sklearn.model_selection import GridSearchCV # get gridsearch with cross validation

In [0]:
#provide GB hyperparameters
gb_hyperparameters = {
    "n_estimators": [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [1, 3, 5]
}

nfolds = 10 #number of folds for CV
gbClassifier = ensemble.GradientBoostingClassifier() #initialise GB classifier

# create Grid search object
gs_gb_clf = GridSearchCV(gbClassifier, gb_hyperparameters, 
                          n_jobs=1, cv=nfolds,
                          scoring='accuracy')





In [0]:
gs_gb_clf.fit(X_train, y_train) #fit the grid search object

In [0]:
print(gs_gb_clf.best_score_)

In [0]:
print(gs_gb_clf.best_params_)

In [0]:
best_parameters_gs = gs_gb_clf.best_params_ #get the best parameters based on 10x CV grid search

### 6. Initialise a Gradient Boosting Classifier

In [0]:
gbClassifier_best = ensemble.GradientBoostingClassifier(**best_parameters_gs) #intialise GB classifier with best set of parameters


### 7. Model Evaluation on Train data

In [0]:
#perform 10 fold cross validation and plot the CM
CV_predicted = cross_val_predict(gbClassifier_best, X_train, y_train, cv=10) #CV predicted values (training data)
CV_score = cross_val_score(gbClassifier_best, X_train, y_train, cv=10) #CV model score (training data)

print("Cross validation Score on train data: ",CV_score.mean())
print("\n")

print("Confusion matrix on CV predictions (train data)")
print(metrics.confusion_matrix(y_train, CV_predicted)) # confusion matrix on CV predictions (train data)
print("\n")

print("Classification report CV predictions (train data)")
print(metrics.classification_report(y_train, CV_predicted, target_names=['Poisonous', 'Edigble'])) # classification report CV predictions (train data)

### 9. Model Evaluation on Test data

In [0]:
gbClassifier_best_mdl= gbClassifier_best.fit(X_train, y_train) #fit the best GB classifier with training data

y_predicted = gbClassifier_best_mdl.predict(X_test) #Predict the outcomes with best GB classifier for test data

In [0]:
mdl_score = gbClassifier_best_mdl.score(X_test,y_test) #model score (test data)
print ("Model Score on test data:",mdl_score)
print("\n")

print("Confusion matrix (test data)")
print(metrics.confusion_matrix(y_test, y_predicted)) #confusion matrix (test data)
print("\n")

print("Classification report (test data)")
print(metrics.classification_report(y_test, y_predicted, target_names=['Poisonous', 'Edigble'])) # classification report (test data)